# Heather's Capstone Notebook

In [2]:
import pandas as pd
import numpy as np

import wrangle as w
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns


In [49]:
# acquire dataset

df = w.wrangle_walmart()

In [8]:
df.head()

,store,weekly_sales,holiday_flag,temperature,fuel_price,CPI,unemployment,type,store_size,month,year,deflated_series,season
Date,,,,,,,,,,,,,
2010-01-10,5,283178.12,0,71,2.60,212.227,6.768,B,34875,January,2010,1334.32,Winter
2010-01-10,15,566945.95,0,59,2.84,132.757,8.067,B,123737,January,2010,4270.56,Winter
2010-01-10,42,481523.93,0,86,3.00,126.235,9.003,C,39690,January,2010,3814.52,Winter
2010-01-10,33,224294.39,0,91,3.00,126.235,9.265,A,39690,January,2010,1776.81,Winter
2010-01-10,36,422169.47,0,74,2.57,210.440,8.476,A,39910,January,2010,2006.12,Winter


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6435 entries, 2010-01-10 to 2012-12-10
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   store            6435 non-null   int64  
 1   weekly_sales     6435 non-null   float64
 2   holiday_flag     6435 non-null   int64  
 3   temperature      6435 non-null   int64  
 4   fuel_price       6435 non-null   float64
 5   CPI              6435 non-null   float64
 6   unemployment     6435 non-null   float64
 7   type             6435 non-null   object 
 8   store_size       6435 non-null   int64  
 9   month            6435 non-null   object 
 10  year             6435 non-null   int64  
 11  deflated_series  6435 non-null   float64
 12  season           6435 non-null   object 
dtypes: float64(5), int64(5), object(3)
memory usage: 703.8+ KB


## Add Holiday's to column

Goal: Create variable called `holiday_week` with the name of the holiday. Later will break out into `is_superbowl`, `is_laborday`, `is_christmas` etc. no holiday

### Holiday List

Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13

Labor Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13

Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13

Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

In [18]:
# get value counts of weeks with holidays and weeks without
df.holiday_flag.value_counts()

0    5985
1     450
Name: holiday_flag, dtype: int64

There's already a column called holiday flag. 0 no holiday 1 holiday. Since we already have this information, we need to know which of these weeks belong to what projects. 

In [47]:
# OOOHHH Those weeks are already there. So Need to filter for those 

christmases = ['2010-12-31', '2011-12-30', '2012-12-28']

super_bowles = ['2010-02-12', '2011-02-11', '2012-02-10']

labor_days = ['2010-09-10', '2011-09-09', '2012-09-07']

thanksgivings = ['2010-11-26', '2011-11-25', '2012-11-23']


In [44]:
# Create loopy deal

def add_which_holiday(dates_list, )
    dates_list = [dt.datetime.strptime(date, "%Y-%m-%d").date() for date in christmases]


In [45]:
dates_list

[datetime.date(2010, 12, 31),
 datetime.date(2011, 12, 30),
 datetime.date(2012, 12, 28)]

In [46]:
df[df.index.isin(dates_list) == True]

,store,weekly_sales,holiday_flag,temperature,fuel_price,CPI,unemployment,type,store_size,month,year,deflated_series,season
Date,,,,,,,,,,,,,
2010-12-31,28,1090558.57,1,45,3.15,127.088,14.313,A,206302,December,2010,8581.15,Winter
2010-12-31,39,1230012.16,1,52,2.94,210.182,8.476,A,184109,December,2010,5852.12,Winter
2010-12-31,9,459770.85,1,45,2.94,214.927,6.560,B,125833,December,2010,2139.20,Winter
2010-12-31,43,534740.30,1,48,2.94,203.418,10.210,C,41062,December,2010,2628.78,Winter
2010-12-31,3,382677.76,1,53,2.94,214.699,7.564,B,37392,December,2010,1782.39,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-12-30,27,1650604.60,1,37,3.57,140.587,7.906,A,204184,December,2011,11740.77,Winter
2011-12-30,17,777207.30,1,26,3.12,130.071,6.617,B,93188,December,2011,5975.25,Winter
2011-12-30,33,215359.21,1,51,3.43,130.071,8.010,A,39690,December,2011,1655.70,Winter


In [41]:
df.index

DatetimeIndex(['2010-01-10', '2010-01-10', '2010-01-10', '2010-01-10',
               '2010-01-10', '2010-01-10', '2010-01-10', '2010-01-10',
               '2010-01-10', '2010-01-10',
               ...
               '2012-12-10', '2012-12-10', '2012-12-10', '2012-12-10',
               '2012-12-10', '2012-12-10', '2012-12-10', '2012-12-10',
               '2012-12-10', '2012-12-10'],
              dtype='datetime64[ns]', name='Date', length=6435, freq=None)

In [50]:
sns.displot(year)

NameError: name 'year' is not defined